In [24]:
!pip install langchain
!pip install faiss-cpu
!pip install sentence-transformers
!pip install transformers


In [ ]:
!pip install langchain-community


In [5]:
import json


In [25]:
import json

# Load the JSON data
with open('/content/data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the first district for preview
print(data[0]['district'])  # Check first district's name


پنجاب


In [12]:
!pip install docarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 7.0 MB/s eta 0:00:00


In [52]:
# Extract relevant documents (service names and descriptions)
documents = [
    f"Service Name: {entry['service_name']}\nDescription: {entry['description']}"
    for entry in data if 'service_name' in entry
]

# Check the first few documents to confirm extraction
print(documents[:2])


['Service Name: کریکٹر سرٹیفیکیٹ\nDescription: با ہر کے ممالک جانے کے لیے، باہر کے مختلف اداروں میں ملازمت حاصل کرنے کےلیے اور وطن بدلی کے لیے پولیس کریکٹر سرٹیفیکیٹ کی ضرورت پڑتی ہے۔', 'Service Name: پولیس ویریفیکیشن\nDescription: پاکستان کے مختلف اداروں میں ملازمت کی درخواست کے لیے پولیس ریکارڈ کی عمومی تصدیق ضروری ہوتی ہے۔']


In [53]:
from sentence_transformers import SentenceTransformer

# Use a multilingual model that supports Urdu
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# Generate embeddings for the documents
document_embeddings = embedding_model.encode(documents)

# Check the first few embeddings
print(document_embeddings[:2])  # Print the first two embeddings for inspection


[[ 0.01890766  0.0707453   0.02207121 ...  0.03602402 -0.02189512
  -0.00826337]
 [ 0.03245357  0.04563726 -0.00271938 ...  0.04700581 -0.0165838
  -0.01438065]]


In [122]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
import json

# Step 1: Load and preprocess data
file_path = "data.json"  # Replace with your JSON file path
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

texts = []
for entry in data:
    district = entry.get("district", "")
    details = entry.get("details", {})
    text = f"District: {district}\n"
    for key, value in details.items():
        text += f"{key.capitalize()}: {value}\n"
    texts.append(text)

splitter = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=100)
documents = splitter.create_documents(texts)

# Step 2: Initialize embeddings and vector store
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Step 3: Initialize open-source LLM
model_name = "EleutherAI/gpt-neo-125M"  # Replace with better Urdu model if available
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,  # Specifies tokens to generate instead of total length
    pad_token_id=tokenizer.eos_token_id  # Prevent padding issues
)
llm = HuggingFacePipeline(pipeline=generator)

# Step 4: Define prompt for question answering
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Answer the question using the context below. Only use the provided context:\n\n{context}\n\nQuestion: {question}"
)

# Step 5: Setup Retrieval-based QA Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": qa_prompt},
    return_source_documents=False
)


# # Step 6: Interaction loop
# print("Chatbot setup complete. You can start querying.")
# while True:
#     query = input("\nUser: ")
#     if query.lower() in ["exit", "quit"]:
#         print("Exiting chatbot. Goodbye!")
#         break
#     response = qa_chain.invoke({"query": query})  # Use invoke instead of run
#     result = response["result"] if "result" in response else "No answer found."
#     print("\nBot:", result)



Device set to use cpu


Chatbot setup complete. You can start querying.

User: لاہور کے پولیس خدمت مرکز کا پتہ کیا ہے؟

Bot: Answer the question using the context below. Only use the provided context:

District: لاہور 5
Address: پولیس خدمت مرکز سروس سیکٹر A، بحریہ ٹاؤن، لاہور
Email: pkm.lahore@punjabpolice.gov.pk
Phone: 04235454046
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 19/6/2020
Location: نقشہ پر لوکیشن

District: لاہور 4
Address: پولیس خدمت مرکز گیٹ نمبر 5 گریٹر اقبال پارک نزد لیڈی ولنگڈن لاہور
Email: pkm.lahore@punjabpolice.gov.pk
Phone: 04299214847
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 07/17/2019
Location: نقشہ پر لوکیشن

District: لاہور 8
Address: ای الخدمت مرکز، لاہور II ٹاؤن ہال لوئر مال، انارکلی، لاہور، پنجاب
Email: pkm.lahore@punjabpolice.gov.pk
Phone: +92-42-99214572
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 20/8/2020
Location: نقشہ پر لوکیشن

District: لاہور 2
Address: لبرٹی پ

KeyboardInterrupt: Interrupted by user

In [125]:
import os  # اسکرین صاف کرنے کے لیے
import re  # اردو ٹیکسٹ کی تصدیق کے لیے

# اردو ٹیکسٹ کی تصدیق کرنے کا فنکشن
def is_urdu_text(input_text):
    urdu_pattern = re.compile(r'^[\u0600-\u06FF\s]+$')
    return bool(urdu_pattern.match(input_text))

# اسکرین صاف کرنے کا فنکشن
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# چیٹ بوٹ کا انٹرایکشن لوپ
clear_screen()  # اسکرین صاف کریں
print("بوٹ: السلام علیکم! میں آپ کی کیسے مدد کرسکتا ہوں؟")  # اسلامی سلام

while True:
    query = input("\nآپ: ")  # صارف سے ان پٹ لینا

    # چیٹ بوٹ بند کرنے کے لیے
    if query.strip() in ["خروج", "بند", "اختتام"]:
        print("بوٹ: اللہ حافظ! دعا میں یاد رکھیں۔")
        break

    # غیر اردو انپٹ کی ہینڈلنگ
    if not is_urdu_text(query):
        print("\nبوٹ: معاف کیجیے! یہ بوٹ صرف اردو میں سوالات قبول کرتا ہے۔")
        continue

    # سوال پر پراسیسنگ
    try:
        response = qa_chain.invoke({"query": query})  # سوال کا جواب حاصل کریں
        result = response["result"] if "result" in response else "معاف کیجیے، اس سوال کے لیے مرکز کی معلومات میسر نہیں۔"

        if not result.strip():
            print("\nبوٹ: معاف کیجیے، میری معلومات مخصوص ویب سائٹ کی حد تک محدود ہے۔")
        else:
            print(f"\nبوٹ: {result}")
    except Exception as e:
        print("\nبوٹ: معاف کیجیے، کچھ غلط ہوا۔ براہ کرم دوبارہ کوشش کریں۔")


بوٹ: السلام علیکم! میں آپ کی کیسے مدد کرسکتا ہوں؟

آپ: لاہور کے پولیس خدمت مرکز کے اوقات کیا ہیں؟

بوٹ: Answer the question using the context below. Only use the provided context:

District: لاہور 5
Address: پولیس خدمت مرکز سروس سیکٹر A، بحریہ ٹاؤن، لاہور
Email: pkm.lahore@punjabpolice.gov.pk
Phone: 04235454046
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 19/6/2020
Location: نقشہ پر لوکیشن

District: لاہور 4
Address: پولیس خدمت مرکز گیٹ نمبر 5 گریٹر اقبال پارک نزد لیڈی ولنگڈن لاہور
Email: pkm.lahore@punjabpolice.gov.pk
Phone: 04299214847
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 07/17/2019
Location: نقشہ پر لوکیشن

District: لاہور 1
Address: ڈی جی آپریشنز آفس، لاہور
Email: pkm.lahore@punjabpolice.gov.pk
Phone: 04299214847
Operational_info: پیر تا ہفتہ - صبح 9 تا شام 5 بجے تک، اتوار - بند
Opening_date: 10/12/2015
Location: نقشہ پر لوکیشن

District: لاہور 2
Address: لبرٹی پارکنگ کے قریب، گلبرگ 3، لاہور
Email: pkm.

**Next Level**